In [ ]:
# Import used packages
import numpy as np
import matplotlib.pyplot as plt
import random

In [ ]:
# Create positions in rectangle of xlen x ylen
def randompos(nodes, xlen, ylen):
    pos = []
    for i in range(nodes):
        pos.append([xlen*random.uniform(0, 1), ylen*random.uniform(0, 1)])
    return pos

# Link nodes
def linknodes(pos, radius):
    links = []
    sets = []
    for i in range(len(pos)):
        linksCurrent = []
        for j in range(len(pos)):
            distance = (pos[i][0] - pos[j][0])**2 + (pos[i][1] - pos[j][1])**2
            if distance <= radius**2:
                if i != j:
                    linksCurrent.append(j)
                    if (i, j) not in sets:
                        sets.append((i, j))
        links.append(linksCurrent)
    return links, sets

# Create network in a dictionary structure
def createnetwork(nodes, radius, xlen, ylen):
    Network = {'pos': randompos(nodes, xlen, ylen)}
    Network['links'], Network['sets'] = linknodes(Network['pos'], radius)
    return Network

# plot nodes
def plotnodes(pos, ax, sizeNodes, colorNodes):
    ax.scatter([item[0] for item in pos], [item[1] for item in pos], s= sizeNodes, c= colorNodes, zorder= 2)
    
#plot edges
def plotedges(pos, ax, sets, colorEdges, alphaEdges):
    for linkedNodes in sets:
        ax.plot([pos[linkedNodes[0]][0], pos[linkedNodes[1]][0]], [pos[linkedNodes[0]][1], pos[linkedNodes[1]][1]], color = colorEdges, alpha= alphaEdges, zorder= 1)

# Plot a random graph
def plotrandomgraph(nodes, radius, xlen, ylen, sizeNodes, colorNodes, colorEdges, alphaEdges):
    Network = createnetwork(nodes, radius, xlen, ylen)
    fig, ax = plt.subplots(figsize = [6*xlen, 6*ylen])
    plt.axis('off')
    plotedges(Network['pos'], ax, Network['sets'], colorEdges, alphaEdges)
    plotnodes(Network['pos'], ax, sizeNodes, colorNodes)

In [ ]:
nodes = 200
xlen = 1
ylen = 1
radius = 0.1
sizeNodes = 80
colorNodes = 'palegreen'
colorEdges = 'grey'
alphaEdges = 0.4

plotrandomgraph(nodes, radius, xlen, ylen, sizeNodes, colorNodes, colorEdges, alphaEdges)

In [ ]:
def createstartsituationSIR(nodes, radius, sickPerc, xlen, ylen, colors):
    Network = createnetwork(nodes, radius, xlen, ylen)
    Network['currentSituation'] = [0]*nodes
    Network['colorSituation'] = [colors[0]]*nodes
    setSick = set([0])
    while len(setSick) < sickPerc*nodes:
        currentsetSick = setSick
        for i in currentsetSick:
            setSick = setSick|set(Network['links'][i])
        if len(setSick) == len(currentsetSick):
            break
    for i in setSick:
        Network['currentSituation'][i] = 1
        Network['colorSituation'][i] = colors[1]
    return Network

def updateSIR(Network, beta, gamma, colors):
    Network['newSituation'] = []
    for i in range(len(Network['currentSituation'])):
        if Network['currentSituation'][i] == 0:
            Phealthy = 1
            for j in Network['links'][i]:
                if Network['currentSituation'][j] == 1:
                    Phealthy *= (1 - beta)
            PSick = 1 - Phealthy
            randomN = random.uniform(0, 1)
            if randomN < PSick:
                Network['newSituation'].append(1)
                Network['colorSituation'][i] = colors[1]
            else:
                Network['newSituation'].append(Network['currentSituation'][i])
        elif Network['currentSituation'][i] == 1:
            randomN = random.uniform(0, 1)
            if randomN < gamma:
                Network['newSituation'].append(2)
                Network['colorSituation'][i] = colors[2]
            else:
                Network['newSituation'].append(Network['currentSituation'][i])
        else:
            Network['newSituation'].append(Network['currentSituation'][i])
    Network['currentSituation'] = Network['newSituation']
    return Network

def spreaddiseaseSIR(Network, NLoops, beta, gamma, colors, xlen, ylen, sizeNodes, colorNodes, colorEdges, alphaEdges):
    plotnetwork(Network, sizeNodes, colorNodes, colorEdges, alphaEdges)
    for i in range(NLoops):
        Network = updateSIR(Network, beta, gamma, colors)
        plotnetwork(Network, sizeNodes, colorNodes, colorEdges, alphaEdges)

def plotnetwork(Network, sizeNodes, colorNodes, colorEdges, alphaEdges):
    fig, ax = plt.subplots(figsize = [6*xlen, 6*ylen])
    plt.axis('off')
    plotedges(Network['pos'], ax, Network['sets'], colorEdges, alphaEdges)
    plotnodes(Network['pos'], ax, sizeNodes, Network['colorSituation'])

In [ ]:
# Plot of starting situation
nodes = 250
xlen = 1
ylen = 1
radius = 0.1
sizeNodes = 80
colors = ['palegreen', 'tomato', 'lightskyblue']
alphaEdges = 0.4
sickPerc = 0.05

Network = createstartsituationSIR(nodes, radius, sickPerc, xlen, ylen, colors)
plotnetwork(Network, sizeNodes, colorNodes, colorEdges, alphaEdges)

In [ ]:
# Network at all timesteps (~30 seconds to run)
nodes = 200
xlen = 1
ylen = 1
radius = 0.12
sizeNodes = 80
alphaEdges = 0.4
colors = ['palegreen', 'tomato', 'lightskyblue']
sickPerc = 0.05
beta = 0.6
gamma = 0.2
NLoops = 6

Network = createstartsituationSIR(nodes, radius, sickPerc, xlen, ylen, colors)
spreaddiseaseSIR(Network, NLoops, beta, gamma, colors, xlen, ylen, sizeNodes, colorNodes, colorEdges, alphaEdges)